In [77]:
import os
import re
import json
import time
import hashlib
import random
import ssl
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [123]:
IMAGEDIR='images'
PROFILES='scam'


extractors = {'username': re.compile('username: ([^\n]+)'),
              'name': re.compile('\Wname: ([^\n]+)'),
              'age': re.compile('\Wage: ([^\n]+)'),
              'location': re.compile('\Wlocation: ([^\n]+)'),
              'ethnicity': re.compile('\Wethni?city: ([^\n]+)'),
              'occupation': re.compile('\Woccupation: ([^\n]+)'),
              'status': re.compile('\Wmarital status: ([^\n]+)'),
              'phone': re.compile('\Wtel: ([^\n]+)'),
              'inet': re.compile('\WIP address: ([^\n]+)'),
              'email': re.compile('\Wemail: ([^\n]+)'),
              'description': re.compile('\Wdescription:([\n\w\W]+)\Wmessage:'),
              'messages': re.compile('\Wmessage:([\n\w\W]+)\WWHY IS'),
              'justifications': re.compile('\WWHY IS IT A SCAM / FAKE:([\n\w\W]+?)(?=This post)')}



In [6]:
def save_image(url):
    """ Take a URL, generate a unique filename, save 
        the image to said file and return the filename."""
    ext = url.split('.')[-1]
    filename = IMAGEDIR+os.sep+hashlib.md5(url.encode('utf-8')).hexdigest()+'.'+ext
    if os.path.exists(filename):
        return filename
    try:
        content = urlopen(url).read()
        f = open(filename,'wb') 
        f.write(content)
        f.close()
    except:
        return None
    return filename 


In [124]:
def scrape_profile(inhandle,year,month):
  """Scrape an input scamdiggers page for the profile content
  of the scammer. """
  #Read file
  html = inhandle.read()
  soup = BeautifulSoup(html, 'html.parser')

  #Find all the profiles of the scammers
  contents = soup.find_all('div', {'class':'entry-content'})

  profile = {}

  #Fill in known info from URL
  #profile['year_reported'] = year
  #profile['month_reported'] = month

  #Extract and download images.
  #profile['images'] = [save_image(img['src']) for img in content.findAll('img')]

  #Get link for all the scammers
  for content in contents:
     link = content.find('a', class_='more-link')
     if link:
        url = link.get('href')
        print("URL:", url)
        urltemp = urlopen(url,context=ssl._create_unverified_context())
        htmltemp = urltemp.read()
        souptemp = BeautifulSoup(htmltemp, 'html.parser')

        #Find main page content
        contenttemp = souptemp.find_all('div', {'class':'entry-content'})
        
        text = ""
        for div in contenttemp:
          paragraphs = div.find_all('p')
          for paragraph in paragraphs:
            text += paragraph.get_text().strip() + "\n"
        print(text)


        #Parse information from text
        for key in extractors:
          match = extractors[key].search(text)
          if match:
            matchtext = match.group(1).strip()
          #if key in ['justifications','messages']:
            #vals = matchtext.split('\n')
          #else:
            vals = matchtext
          profile[key] = vals 
        print(profile)
        outfile=profile['username']+'.json'
        json.dump(profile, open(outfile,'w'))
  return len(contents)

  #Parse annotations
  #content = soup.find('div', {'class':'entry-utility'})
  #profile['tags']   = [node.get_text() for node in content.findAll('a', {'rel':'tag'})]
  #profile['gender'] = 'female' if 'Female profiles' in profile['tags'] else 'male'

  #Save output
  



In [8]:
def enumerate_profiles(inhandle, page):
  """ Extract all the profile page links from
  this index page. """
  html = inhandle.read()
  soup = BeautifulSoup(html, 'html.parser')
  
  urls = [ node.find('a')['href'] for node in soup.findAll('h1',  {'class':'entry-title'})]
  return urls

In [104]:
def gather_all_profiles(year, month):
  """ Walk the index pages, harvesting the profile URLs,
  and then download and process all the profiles stored 
  under this year and month. """
  page = 1
  urls = []

  print("{}-{} : Begin indexing.".format(year, month))

  while (page > 0):
    urlstring = "http://scamdigger.com/{}/{}/page/{}".format(year,month,page)   
    jitter = random.choice([0,1])
    try:
      urlhandle = urlopen(urlstring)
      scrape_profile(urlhandle,year,month)
      #urls += enumerate_profiles(urlhandle, page)
      time.sleep(1+jitter)
      page += 1
    except:
      page = 0

  print("{}-{}-{} : {} profiles".format(year,month,page,num))

  print("{}-{} : complete.".format(year,month))


In [127]:
def scrape(startyear, startmonth, endyear, endmonth):
  """ Walk the database through the defined ranges,
  downloading everything. """
  year = startyear
  month = startmonth
  while (not (year == endyear and month == endmonth)):
    ys = "{}".format(year)
    ms = "{:02d}".format(month)
    gather_all_profiles(ys,ms) 
    if month == 12:
      year += 1
      month = 0
    month += 1

In [128]:
scrape(2016,6,2017,4)

2016-06 : Begin indexing.
URL: https://scamdigger.com/2016/06/olivier-bavinchov-generalolivier900gmail-com/
username: generalolivier
name: Olivier Bavinchov
email: generalolivier900@gmail.com
age: 58
location: Kabul, Afghanistan
ethnicity: mixed
occupation: Commander
marital status: divorced
IP address: 41.218.251.239
description:
Am General Olivier i have been divorced years back. Am looking for a Good woman i can spend the rest of my life with.
message: –
WHY IS IT A SCAM / FAKE:
IP is Ghana
IP doesn’t correspond location
Pictures are stolen from a known source
Pictures were used in scams before
Also see here:
https://www.romancescam.com/forum/viewtopic.php?t=91976
This post was viewed 1822 times

{'username': 'generalolivier', 'name': 'Olivier Bavinchov', 'age': '58', 'location': 'Kabul, Afghanistan', 'ethnicity': 'mixed', 'occupation': 'Commander', 'status': 'divorced', 'phone': 'divorced', 'inet': '41.218.251.239', 'email': 'generalolivier900@gmail.com', 'description': 'Am General

username: frederick40
name: Frederick Brown
email: frederickbrown27@gmail.com
alternative email: ya.frederickbrown@yandex.com
age: 40
location: New York, United States
ethnicity: white
occupation: financial consultant
marital status: widowed
IP address: 104.236.0.232
description:
my name is Frederick. im 40 years single man. My hobbies include reading books, watching movies and traveling to tourist centers.
message:
Hi my name is Frederick i see your profile and it attract me to write to you. im 40 years single man and I would like to establish friendship with you so that we can know each other.

WHY IS IT A SCAM / FAKE:
IP is a proxy
IP doesn’t correspond location
Registered on different sites with different pictures
Mass-mailing
This post was viewed 1588 times

{'username': 'frederick40', 'name': 'Frederick Brown', 'age': '40', 'location': 'New York, United States', 'ethnicity': 'white', 'occupation': 'financial consultant', 'status': 'widowed', 'phone': 'widowed', 'inet': '104.236.0